In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Hyperparameters and constants
IMG_SIZE = (128, 128)
BATCH_SZ = 32
NUM_EPOCHS = 10

def prepare_data(directory_path):
    """
    Prepares and preprocesses image data for training and validation.

    Args:
        directory_path (str): Path to the dataset.

    Returns:
        tuple: Training and validation data generators.
    """
    data_augmentor = ImageDataGenerator(
        rescale=1.0 / 255, 
        validation_split=0.2
    )

    # Training data generator
    train_data = data_augmentor.flow_from_directory(
        directory_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SZ,
        class_mode='categorical',
        subset='training'
    )

    # Validation data generator
    val_data = data_augmentor.flow_from_directory(
        directory_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SZ,
        class_mode='categorical',
        subset='validation'
    )

    return train_data, val_data

def build_cnn_model(class_count):
    """
    Builds and compiles a CNN model for classification.

    Args:
        class_count (int): Number of output classes.

    Returns:
        keras.Model: Compiled CNN model.
    """
    cnn_model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(class_count, activation='softmax')
    ])

    cnn_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return cnn_model

# Dataset directory
dataset_path = r'C:\Users\moham\OneDrive\Desktop\internship\task 4\PRODIGY_ML_04-main\train_03'

# Load and preprocess the data
training_data, validation_data = prepare_data(dataset_path)

# Determine the number of output classes
output_classes = len(training_data.class_indices)

# Build and compile the CNN model
gesture_model = build_cnn_model(output_classes)

# Train the model and validate on validation data
training_history = gesture_model.fit(
    training_data,
    epochs=NUM_EPOCHS,
    validation_data=validation_data
)

# Save the trained model to a file
model_file = 'gesture_recognition_model.h5'
gesture_model.save(model_file)
print(f"Model saved as {model_file}")


Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.


C:\Users\moham\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\moham\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 52s 984ms/step - accuracy: 0.6581 - loss: 1.0764 - val_accuracy: 0.9850 - val_loss: 0.0357
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 460ms/step - accuracy: 1.0000 - loss: 5.3761e-04 - val_accuracy: 0.9900 - val_loss: 0.0253
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 451ms/step - accuracy: 1.0000 - loss: 6.9756e-05 - val_accuracy: 0.9875 - val_loss: 0.0266
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 22s 448ms/step - accuracy: 1.0000 - loss: 3.4451e-05 - val_accuracy: 0.9900 - val_loss: 0.0241
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 452ms/step - accuracy: 1.0000 - loss: 2.1313e-05 - val_accuracy: 0.9925 - val_loss: 0.0230
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 22s 446ms/step - accuracy: 1.0000 - loss: 1.4129e-05 - val_accuracy: 0.9925 - val_loss: 0.0238
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 24s 471ms/step - accuracy: 1.0000 - loss: 1.0697e-05 - val_accuracy: 0.9925 - val_loss: 0.0245
Epoch 8/10
43/50 ━━━━━━━━━━━━━━━━━━━━ 2s 404ms/step - accuracy: 1.0000 - 

KeyboardInterrupt: 

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from skimage.io import imread
import matplotlib.pyplot as plt
import tensorflow as tf

# Load the pre-trained model
gesture_recognition_model = tf.keras.models.load_model('hand_gesture_model.h5')

def identify_hand_gesture(img_path):
    """
    Predict the hand gesture from the given image.

    Args:
        img_path (str): File path of the input image.

    Returns:
        str: Predicted gesture class label.
    """
    # Load and preprocess the image
    input_img = image.load_img(img_path, target_size=IMG_SIZE)
    input_array = image.img_to_array(input_img)
    input_array = np.expand_dims(input_array, axis=0) / 255.0

    # Predict using the model
    predictions = gesture_recognition_model.predict(input_array)
    predicted_index = np.argmax(predictions)

    # Map the prediction to class labels
    gesture_classes = list(training_data.class_indices.keys())
    return gesture_classes[predicted_index]

# Define the path to the test image
test_image_path = r'C:\Users\moham\OneDrive\Desktop\internship\task 4\PRODIGY_ML_04-main\test_03\frame_02_07_0009.png'

# Predict the gesture for the given image
predicted_gesture_label = identify_hand_gesture(test_image_path)

# Display the input image
test_img = imread(test_image_path)
plt.imshow(test_img)
plt.title(f"Predicted Gesture: {predicted_gesture_label}")
plt.axis('off')
plt.show()

print(f"Predicted hand gesture: {predicted_gesture_label}")
